In [ ]:
# Para centralizar as imagens:
from IPython.core.display import display, HTML
HTML("""<style>.output_png{display: table-cell;    text-align: center;    vertical-align: middle;}</style>""")
import numpy as np

# **Resolução de um Problema de Negócio do Walmart**

**Título:** *Store Sales Forecasting* - Previsão de Vendas das Lojas

**Referência:** https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/overview/evaluation

**Objetivos:** 

1) Prever as vendas das lojas usando dados históricos

2) Estimar o impacto de uma semana com feriado nas vendas semanais

               
**Métrica:** A métrica de avaliação da eficiência do modelo é:

![\Large%20WMAE=\frac{1}{\sum{w_i}}\sum_{i=1}^n%20w_i|y_i-\hat{y}_{i}|](https://latex.codecogs.com/svg.latex?\Large%20WMAE=\frac{1}{\sum{w_i}}\sum_{i=1}^n%20w_i|y_i-\hat{y}_{i}|)
onde:
* $n$ é o número das linhas
* $\hat{y}_{i}$ é a previsão de venda
* $y^i$ é a venda realizada
* $w_i$ são pesos. $w = 5$ se a semana tiver feriado, $1$ se não tiver

Quanto menor for o valor de WMAE, mais preciso o modelo é em prever as vendas semanais, especialmente nas semanas com feriado.

## **1) Preparação**
**Entendendo o Problema**:  
Primeiramente, é necessário entender qual é tipo de problema em questão para
aplicar a ferramenta (modelo) mais adequada para obtenção da solução. Então, 
vamos dar uma olhada para os dados disponibilizados e o que se pede:

In [ ]:
# ANÁLISE EXPLORATÓRIA DOS DADOS:
    # Importando os dados:
import pandas as pd
features = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/features.csv.zip')
stores = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/stores.csv')
train = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/train.csv.zip')
test = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/test.csv.zip')

Exibindo uma pequena amostra dos dados de treino para o modelo (train.csv) e alguns de seus parâmetros descritivos:

In [ ]:
pd.concat([train.head(3),train.sample(4),train.tail(3)])

In [ ]:
    # Análise Estatística Descritiva dos Dados:
train.describe().T

De acordo com o que se pediu ("[...] project the sales for each department in each store"), os dados de treino disponibilizam as vendas semanais para cada loja e departamento, que estão enumerados. Três coisas chamam a atenção ao analisar este conjunto de dados: 
* 1ª) O formato de data apresenta o dia de término da semana, que aparenta representar sempre as sextas-feiras. A princípio o ideal seria converter para número da semana, já que os resultados retratam vendas semanais;
* 2ª) Há ocorrência de vendas negativas nos dados históricos. Isto pode significar que em determinadas semanas houve mais devoluções do que vendas. O que causa estranheza. Por curiosidade, vamos verificar qual a porcentagem desse conjunto de dados que apresenta vendas semanais negativas:

In [ ]:
round((len(train[train['Weekly_Sales'] < 0])/len(train))*100,2)

0,3% das vendas semanais sendo negativas parece ser razoável.
* 3ª) Parece haver um grande número de *outliers* nos dados de vendas semanais, já que seu valor máximo distoa muito do intervalo interquartil (onde metade dos dados estão) além de que a média é mais que o dobro da mediana (valores muito grandes estão elevando a média). Para visualizar a presença de *outliers*, faz-se o uso do diagrama de caixa, ou boxplot:

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt

plt.figure(figsize=(15,5))
sns.boxplot(x=train['Weekly_Sales'])
plt.show()

In [ ]:
Q1 = train['Weekly_Sales'].quantile(0.25)
Q3 = train['Weekly_Sales'].quantile(0.75)
IQR = Q3 - Q1
Noutliers = (train['Weekly_Sales'] > (Q3 + 1.5 * IQR)).sum()
print(round((Noutliers/len(train))*100,1), '% de outliers no dataset')

Realmente, a hipótese de que existem muitos *outliers* se confirma. Talvez seja ideal removê-los antes de aplicar o modelo, para melhorar sua acurácia. Mas antes vamos ver se eles estão correlacionados com as semanas com feriado:

## 2) Análise das Vendas em Semanas com Feriado:

In [ ]:
outliers = train[train['Weekly_Sales'] > (Q3 + 1.5 * IQR)]
Outliers_Holiday_False_Per100 = int(round(100*outliers[outliers['IsHoliday']==False]['Weekly_Sales'].count()/outliers['Weekly_Sales'].count(),0))
Outliers_Holiday_True_Per100 = int(round(100*outliers[outliers['IsHoliday']==True]['Weekly_Sales'].count()/outliers['Weekly_Sales'].count(),0))
Holiday_False_Per100 = int(round(100*train[train['IsHoliday']==False]['Weekly_Sales'].count()/train['Weekly_Sales'].count(),0))
Holiday_True_Per100 = int(round(100*train[train['IsHoliday']==True]['Weekly_Sales'].count()/train['Weekly_Sales'].count(),0))

print('Porcentagem de semana com feriado vs semana sem feriado - todo o dataset:',Holiday_True_Per100,'% vs',Holiday_False_Per100,'%')
print('Porcentagem de semana com feriado vs semana sem feriado - somente outliers:',Outliers_Holiday_True_Per100,'% vs',Outliers_Holiday_False_Per100,'%')

A princípio, os dados dizem que os **picos** de vendas semanais não ocorrem necessariamente em feriados. Agora, vamos verificar a influência dos feriados de outra forma - comparando a média de vendas das semanas com feriado e sem feriado:

In [ ]:
NoHoliday_Sales = train[train['IsHoliday']==False]['Weekly_Sales'].mean()
Holiday_Sales = train[train['IsHoliday']==True]['Weekly_Sales'].mean()
print('Em média, semanas com feriados vendem',int(round(100*Holiday_Sales/NoHoliday_Sales - 100,0)),'% a mais que semanas sem feriados.')

Surpreendentemente, este não é um valor expressivo, como se esperaria.

Vamos mais a fundo, a fim de identificar a correlação de cada feriado com as médias de vendas semanais para cada ano:

In [ ]:
    # Manipulando as datas para o formato desejado:
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

train.insert(3,'Year',train['Date'].dt.year)
test.insert(3,'Year',test['Date'].dt.year)

train['Date'] = train['Date'].dt.isocalendar().week
test['Date'] = test['Date'].dt.isocalendar().week

train = train.rename(columns={'Date': 'WeekNo'})
test = test.rename(columns={'Date': 'WeekNo'})

    # Identificando as semanas com feriados:
Holidays = {'Super Bowl': ['12-Feb-10', '11-Feb-11', '10-Feb-12', '8-Feb-13'],
            'Labor Day': ['10-Sep-10', '9-Sep-11', '7-Sep-12', '6-Sep-13'],
            'Thanksgiving': ['26-Nov-10', '25-Nov-11', '23-Nov-12', '29-Nov-13'],
            'Christmas': ['31-Dec-10', '30-Dec-11', '28-Dec-12', '27-Dec-13']}
Holidays = pd.DataFrame(Holidays, index=['2010','2011','2012','2013'])
Holidays['Super Bowl'] = pd.to_datetime(Holidays['Super Bowl'], format='%d-%b-%y')
Holidays['Labor Day'] = pd.to_datetime(Holidays['Labor Day'], format='%d-%b-%y')
Holidays['Thanksgiving'] = pd.to_datetime(Holidays['Thanksgiving'], format='%d-%b-%y')
Holidays['Christmas'] = pd.to_datetime(Holidays['Christmas'], format='%d-%b-%y')

Holidays['Super Bowl'] = Holidays['Super Bowl'].dt.isocalendar().week
Holidays['Labor Day'] = Holidays['Labor Day'].dt.isocalendar().week
Holidays['Thanksgiving'] = Holidays['Thanksgiving'].dt.isocalendar().week
Holidays['Christmas'] = Holidays['Christmas'].dt.isocalendar().week

In [ ]:
# Configurando o gráfico:
Weekly_Sales_2010 = train[train.Year==2010]['Weekly_Sales'].groupby(train['WeekNo']).mean()
Weekly_Sales_2011 = train[train.Year==2011]['Weekly_Sales'].groupby(train['WeekNo']).mean()
Weekly_Sales_2012 = train[train.Year==2012]['Weekly_Sales'].groupby(train['WeekNo']).mean()
plt.figure(figsize=(18,6))
sns.lineplot(x=Weekly_Sales_2010.index, y=Weekly_Sales_2010.values)
sns.lineplot(x=Weekly_Sales_2011.index, y=Weekly_Sales_2011.values)
sns.lineplot(x=Weekly_Sales_2012.index, y=Weekly_Sales_2012.values)
plt.grid()
plt.xticks(range(min(train['WeekNo']), max(train['WeekNo'])+1,1))
plt.legend(['2010', '2011', '2012'], loc='best', fontsize=16)
plt.gca().add_artist(plt.legend(['2010', '2011', '2012'], loc='upper left', fontsize=16))
plt.title('Average Weekly Sales per Year\n', fontsize=18)
plt.ylabel('Average Sales', fontsize=16)
plt.xlabel('Week', fontsize=16)
i=0
colors = ['blue', 'lime', 'magenta', 'cyan']
for x in Holidays.iloc[:3,:].drop_duplicates().stack():
    plt.axvline(x, color = colors[i], label = Holidays.columns[i])
    plt.legend(loc='upper center',fontsize=16)
    i = i + 1
plt.show()

Através do gráfico percebe-se que os únicos feriados que realmente causaram impacto significativo nas vendas foram o *Thanksgiving* e o Natal. 

Como parte da avaliação ("*Part of the challenge presented by this competition is modeling the effects of markdowns on these holiday weeks in the absence of complete/ideal historical data*"), vamos verificar e modelar o impacto das promoções nessas datas especiais:

Dando uma olhada no *dataset* de *features* (features.csv):

In [ ]:
pd.concat([features.head(3),features.sample(4),features.tail(3)])

Como foi dito na descrição dos dados, o registro das promoções somente está disponível a partir de novembro de 2011:

"*MarkDown data is only available after Nov 2011, and is not available for all stores all the time. Any missing value is marked with an NA*"

Além disso, percebe-se que o *dataset* de *features* tem dados até 26 de julho de 2013, sendo que os últimos registros do *dataset* de treino ocorrem em 26 de outubro de 2012 (últimos valores de vendas). Então, vamos considerar somente o período entre novembro de 2011 e outubro de 2012 e relacionar com as vendas.

In [ ]:
ftrs = features[['Store','Date','MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','IsHoliday']]
IsHoliday = ftrs['IsHoliday']
ftrs = ftrs.drop(columns=['IsHoliday'])
ftrs.insert(1,'IsHoliday',IsHoliday)

MarkDown_Total = features.loc[:,'MarkDown1':'MarkDown5'].sum(axis=1).rename('MarkDowns_Total')
ftrs.insert(2, 'MarkDown_Total', MarkDown_Total)

ftrs['Date'] = pd.to_datetime(ftrs['Date'])
ftrs.insert(2,'Year',ftrs['Date'].dt.year)
ftrs = ftrs[(ftrs['Date'] >= (pd.to_datetime('11-Nov-11', format='%d-%b-%y'))) & (ftrs['Year'] >= 2011) & (ftrs['Date'] <= (pd.to_datetime('26-Oct-12', format='%d-%b-%y')))]
ftrs['Date'] = ftrs['Date'].dt.isocalendar().week
ftrs = ftrs.rename(columns={'Date': 'WeekNo'})
ftrs.insert(1,'Store_Week_Year',pd.Series(ftrs['Store'].astype(str) + '_' + ftrs['WeekNo'].astype(str) + '_' + ftrs['Year'].astype(str)))
ftrs = ftrs.drop(columns=['Store','WeekNo','Year'])

Store_Week_Year = pd.Series(train['Store'].astype(str) + '_' + train['WeekNo'].astype(str) + '_' + train['Year'].astype(str),name='Store_Week_Year')
trn = pd.concat([Store_Week_Year,train],axis=1)
trn = trn.drop(columns=['Store','WeekNo','Year','Dept'])
trn = trn.groupby(['Store_Week_Year']).agg({'Weekly_Sales':sum,'IsHoliday':'first'})

MarkDown_Sales = pd.merge(ftrs, trn, on = ['Store_Week_Year','IsHoliday'], how = 'inner')

Vamos checar se após filtrar o período ainda há valores faltantes no *dataset*:

In [ ]:
NAs = pd.DataFrame(round(MarkDown_Sales.isnull().sum()/MarkDown_Sales.shape[0], 3)*100,columns=['%NA'])
NAs.T

In [ ]:
MDvsSales = MarkDown_Sales.dropna(0)
DataLoss = MDvsSales.shape[0] - MarkDown_Sales.shape[0]
DataLossP100 = print('Ao eliminar as linhas com valores nulos a perda de informação é de',-DataLoss,'linhas de',
                     MarkDown_Sales.shape[0],', o que equivale a',round(1 - MDvsSales.shape[0]/MarkDown_Sales.shape[0],3)*100,'% do dataset.')

A descrição do problema enfatiza que os valores NA correspondem a valores faltantes e não possivelmente a um valor nulo, ou zero de promoções/ofertas. Sendo assim, apesar de representarem uma parcela significativa do conjunto de dados, foi escolhido eliminar as linhas com NAs, já que substituir estes valores com a média ou mediana da coluna pode enviesar o resultado.

In [ ]:
MDvsSales = MarkDown_Sales.dropna(0)

# Plotando
from scipy import stats as st
sns.set(font_scale=1.3)
fig, axes = plt.subplots(6, 3, figsize=(20,20), tight_layout=True)
fig.suptitle('Markdown-Sales Correlation\n', fontsize=22)
axes[0,0].set_title("\n".join(["All Weeks\n"]), fontsize=16)
axes[0,1].set_title("\n".join(["Only Non-Holidays\n"]), fontsize=16)
axes[0,2].set_title("\n".join(["Only Holidays\n"]), fontsize=16)
MD = ['T', 1, 2, 3, 4, 5]
for i in range(6):
    column_name = 'Z_Score_MD'+str(MD[i])
    MDvsSales.insert(len(MDvsSales.columns),column_name,st.zscore(MDvsSales.iloc[:,i+2]))
    MDvsSales_NoOut = MDvsSales[MDvsSales[column_name].abs() < 3] # Retirando os outliers
    
    data=MDvsSales_NoOut
    sns.regplot(ax=axes[i,0],data=data,x=data.iloc[:,i+2],y='Weekly_Sales',scatter_kws={"color":"gold"},line_kws={"color":"black"})
    r1, p1 = st.pearsonr(data.iloc[:,i+2], data['Weekly_Sales'])
    axes[i,0].text(0.45,0.9,'Plot {:n}'.format(i+1),transform=axes[i,0].transAxes)
    axes[i,0].text(0.83,0.78,'r={:.2f}\np={:.3f}'.format(r1,p1),transform=axes[i,0].transAxes)
    axes[i,0].ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    
    data=MDvsSales_NoOut[MDvsSales_NoOut['IsHoliday']==False]
    sns.regplot(ax=axes[i,1],data=data,x=data.iloc[:,i+2],y='Weekly_Sales',scatter_kws={"color":"gold"}, line_kws={"color":"black"})
    r2, p2 = st.pearsonr(data.iloc[:,i+2], data['Weekly_Sales'])
    axes[i,1].text(0.45,0.9,'Plot {:n}'.format(i+7),transform=axes[i,1].transAxes)
    axes[i,1].text(0.83,0.78,'r={:.2f}\np={:.3f}'.format(r2,p2),transform=axes[i,1].transAxes)
    axes[i,1].ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    
    data=MDvsSales_NoOut[MDvsSales_NoOut['IsHoliday']==True]
    sns.regplot(ax=axes[i,2],data=data,x=data.iloc[:,i+2],y='Weekly_Sales',scatter_kws={"color":"gold"}, line_kws={"color":"black"})
    r3, p3 = st.pearsonr(data.iloc[:,i+2], data['Weekly_Sales'])
    axes[i,2].text(0.45,0.9,'Plot {:n}'.format(i+13),transform=axes[i,2].transAxes)
    axes[i,2].text(0.83,0.78,'r={:.2f}\np={:.3f}'.format(r3,p3),transform=axes[i,2].transAxes)
    axes[i,2].ticklabel_format(axis="x", style="sci", scilimits=(0,0))

Os gráficos explicitam a relação entre a soma das vendas semanais de todas as lojas com a quantidade de cada tipo de *Markdown* aplicado na semana, como também para o total de *Markdowns*. Para isso, foi empregado o modelo de regressão linear com o objetivo de extrair a correlação entre as variáveis. Dispomos das seguintes ferramentas de análise e de suas contribuições pertinentes:
* **Intuição**: É razoável presumir que as vendas aumentam caso o preço diminua;
* **Visualização**: Ao observar a dispersão dos pontos nos gráficos, parece razoável afirmar que a relação entre vendas e ofertas tende a ser linear e a apresentar uma leve covariância positiva na maioria dos casos, apesar da ampla variância dos dados. Essa amplitude pode ser justificada ao considerar que a ofertas influenciariam mais determinados tipos de produtos que outros. O mesmo pode ser dito com relação as marcas. Como não dispomos destes dados aqui, essa poderia ser uma análise futura.
* **Coeficiente de Correlação de Pearson, $\rho$ ("r" no gráfico)**: Analisando os valores de $\rho$ das distribuições que parecem melhor ajustadas, a maior correlação se dá quando se relaciona o total das ofertas em semanas com feriado, o que é exatamente um dos objetivos deste desafio, de forma geral. Neste caso, olhando somente o valor de $\rho$~0.5 aparenta traduzir uma correlação razoável, dada a quantidade de variáveis que envolvem a efetivação de uma compra. Em segundo lugar, nota-se que o *Markdown5* correlaciona-se positivamente com as vendas mais que os outros *Markdowns*, em todos três casos de classificação de semanas.
* **Valor-p**: Para os casos mencionados no item anterior (Plots 6, 12, 13 e 18), os valores-p são praticamente zero, indicando que os respectivos tipos de ofertas têm uma associação com as vendas que é altamente significante.

Como ainda é possível classificar as semanas com feriado em quatro datas distintas (que também se traduzem em diferentes tipos de celebração), vamos fazê-lo a seguir, já que as correlações continuam não sendo tão evidentes.

In [ ]:
sns.set(font_scale=1.3)
fig, axes = plt.subplots(6, 4, figsize=(20,20), tight_layout=True)
fig.suptitle('Markdowns Effect on Holidays Sales\n', fontsize=22)
axes[0,0].set_title("\n".join(["Super Bowl\n"]), fontsize=16)
axes[0,1].set_title("\n".join(["Labor Day\n"]), fontsize=16)
axes[0,2].set_title("\n".join(["Thanksgiving\n"]), fontsize=16)
axes[0,3].set_title("\n".join(["Christmas\n"]), fontsize=16)
MD = ['T', 1, 2, 3, 4, 5]
for i in range(6): 
    data=MDvsSales_NoOut[MDvsSales_NoOut['Store_Week_Year'].str.contains('_6_')]
    sns.regplot(ax=axes[i,0],data=data,x=data.iloc[:,i+2],y='Weekly_Sales',scatter_kws={"color":"gold"},line_kws={"color":"black"})
    r1, p1 = st.pearsonr(data.iloc[:,i+2], data['Weekly_Sales'])
    axes[i,0].text(0.4,0.9,'Plot {:n}'.format(i+19),transform=axes[i,0].transAxes)
    axes[i,0].text(0.75,0.02,'r={:.2f}\np={:.3f}'.format(r1,p1),transform=axes[i,0].transAxes)
    axes[i,0].ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    axes[i,0].set_autoscale_on(True)
    
    data=MDvsSales_NoOut[MDvsSales_NoOut['Store_Week_Year'].str.contains('_36_')]
    sns.regplot(ax=axes[i,1],data=data,x=data.iloc[:,i+2],y='Weekly_Sales',scatter_kws={"color":"gold"}, line_kws={"color":"black"})
    r2, p2 = st.pearsonr(data.iloc[:,i+2], data['Weekly_Sales'])
    axes[i,1].text(0.4,0.9,'Plot {:n}'.format(i+25),transform=axes[i,1].transAxes)
    axes[i,1].text(0.75,0.02,'r={:.2f}\np={:.3f}'.format(r2,p2),transform=axes[i,1].transAxes)
    axes[i,1].ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    
    data=MDvsSales_NoOut[MDvsSales_NoOut['Store_Week_Year'].str.contains('47')]
    sns.regplot(ax=axes[i,2],data=data,x=data.iloc[:,i+2],y='Weekly_Sales',scatter_kws={"color":"gold"}, line_kws={"color":"black"})
    r3, p3 = st.pearsonr(data.iloc[:,i+2], data['Weekly_Sales'])
    axes[i,2].text(0.4,0.9,'Plot {:n}'.format(i+31),transform=axes[i,2].transAxes)
    axes[i,2].text(0.75,0.02,'r={:.2f}\np={:.3f}'.format(r3,p3),transform=axes[i,2].transAxes)
    axes[i,2].ticklabel_format(axis="x", style="sci", scilimits=(0,0))
    
    data=MDvsSales_NoOut[MDvsSales_NoOut['Store_Week_Year'].str.contains('51')]
    sns.regplot(ax=axes[i,3],data=data,x=data.iloc[:,i+2],y='Weekly_Sales',scatter_kws={"color":"gold"}, line_kws={"color":"black"})
    r4, p4 = st.pearsonr(data.iloc[:,i+2], data['Weekly_Sales'])
    axes[i,3].text(0.4,0.9,'Plot {:n}'.format(i+37),transform=axes[i,3].transAxes)
    axes[i,3].text(0.75,0.02,'r={:.2f}\np={:.3f}'.format(r4,p4),transform=axes[i,3].transAxes)
    axes[i,3].ticklabel_format(axis="x", style="sci", scilimits=(0,0))

Aqui podemos notar que as correlações se tornaram mais expressivas em alguns casos, possibilitando conhecer melhor o impacto de cada tipo de oferta em cada feriado.
De modo geral, ou seja, considerando o total de ofertas, temos consideráveis associações em todos os feriados, sendo o período entre o Dia de Ação de Graças o mais afetado, seguido pelo Dia do Trabalho, *Super Bowl* e Natal. 

A primeira vista, a sequência pode não fazer sentido já que o Natal está em último. Porém, uma hipótese que poderia ser melhor estudada no futuro seria que o período do Natal, por ser tradicionalmente a época que está mais relacionada a compras, é portanto, naturalmente potencializada, gerando o maior pico de venda e consequentemente uma maior capacidade das lojas em absorver esta demanda. Devido a esse fato, pode não ser interessante alavancar ainda mais esse período já esponteneamente vendável e saturar as lojas ou reduzir o faturamento. 
Já os demais feriados seguem uma lógica de acordo com a tradição de vendas americana.

Mais especificamente, para cada semana de feriado abaixo, seguem os tipos de ofertas que denotam evidente correlação (maior que moderada), do mais forte para o mais fraco:
* ***Super Bowl***: *MarkDown* 3, 5, e 2;
* ***Labor Day***: *MarkDown* 5, 4 e 1;
* ***Thanksgiving***: *MarkDown* 3;
* ***Christmas***: *MarkDown* 3.

Pelas informações disponibilizadas, não é possível saber o caráter de cada tipo de *MarkDown*, se são frutos de escolhas (eventuais promoções/descontos) ou de consequências (concorrência, custo de estocagem, produtos danificados etc). Sendo assim, recomendar determinado tipo de *MarkDown* a um período de feriado específico é incerto a princípio. Supondo que sejam somente do tipo promoções/descontos, a priorização destacada logo acima seria indicada.

Partimos agora para o desenvolvimento do modelo preditivo:

## 3) Desenvolvimento do Modelo Preditivo:

Como vimos anteriormente, por se tratar de um problema supervisionado de regressão (vide train.csv), com variáveis tanto categóricas (Store, Dept, Week, IsHoliday...) quanto contínuas (features, Weekly_Sales...), apresentar um comportamento não linear (vide gráfico "Average Weekly Sales per Year) e de fina granularidade de dados (45 Stores > 99 Depts > 52 Weeks), aplicar algum modelo de aprendizado de máquina baseado no algoritmo **Árvore de Decisão** parece ser uma boa aposta inicialmente, já que ele costuma a lidar bem em *datasets* com este conjunto de particularidades.

A depender dos parâmetros configurados, esse algoritmo pode tender ao fenômeno de *overfitting*, ou seja, ajustar especificamente ao *dataset* de treino e não de uma forma generalizada, perdendo acurácia a receber quaisquer dados diferentes ao que o modelo foi treinado. Portanto, iremos utilizar o algoritmo denominado ***Random Forest*** (ou Floresta Aleatória), o qual também é baseado em **Árvore de Decisão**, porém minimiza a chance de ocorrer *overfitting* ao incorporar múltiplas árvores de decisão em paralelo e tomar o valor mais frequente dos resultados. Mesmo assim, há algum risco de sobreajustar o modelo, então iremos verificar a acurácia em dados de teste adiante.

Feitas as considerações iniciais, vamos implementá-lo:

O primeiro a se fazer é conhecer e selecionar somente as variáveis realmente relevantes para a predição das vendas semanais. Essa etapa é importante pois faz com que o modelo seja mais fácil de compreender, além de proporcioná-lo um melhor desempenho, tanto com relação à acurácia quanto ao tempo de execução.

In [ ]:
train_ftrs = features
train_ftrs['Date'] = pd.to_datetime(train_ftrs['Date'])
train_ftrs.insert(2,'Year',train_ftrs['Date'].dt.year)
train_ftrs = train_ftrs[train_ftrs['Date'] <= (pd.to_datetime('26-Oct-12', format='%d-%b-%y'))]
train_ftrs['Date'] = train_ftrs['Date'].dt.isocalendar().week
train_ftrs = train_ftrs.rename(columns={'Date': 'WeekNo'})
print(len(stores.columns.append(train_ftrs.columns).append(train.columns).drop_duplicates()),'variáveis')
NAs = pd.DataFrame(round(train_ftrs.isnull().sum()/train_ftrs.shape[0], 3)*100,columns=['%NA'])
NAs.T

**Análise das variáveis**:
    
No total temos 17 variáveis. Como visto anteriormente, boa parte das *features* "*MarkDown*" têm valores faltantes. Como para o treino é essencial que não haja dados nulos, penso em três opções: 
1.     Desconsiderar as variáveis *MarkDown* para a predição de *Weekly_Sales* > Perda de poder preditivo, já que foi visto uma correlação entre as variáveis acima, especialmente em semanas de feriado, onde a penalização da métrica WMAE é maior;
2.  Eliminar as linhas onde há dados nulos > eliminar quase 75% do *dataset*, perdendo informações das outras variáveis para compor a predição;
3.  Substituir os valores pela média ou mediana do respectivo *MarkDown* > resultados enviesados por essa manipulação, já que mais da metade das variáveis são nulas.

A opção que parece mais razoável no momento é desconsiderar as variáveis *MarkDown* para o treino do modelo. Considerando que os registros de remarcação sejam mais constantes a partir de novembro de 2011, é possível incorporar estas variáveis na realimentação do modelo com novos dados históricos.

Outra modificação que a princípio parece ser interessante é incluir uma variável para cada feriado, já que foi visto que cada um deles impacta de forma diferente as vendas semanais, com o Thanksgiving e o *Christmas* apresentando os maiores picos de venda.

Já a normalização das variáveis não é necessária quando se utiliza algoritmos baseados em Arvore de Decisão.

In [ ]:
# Criando variáveis para cada feriado
train_ftrs = train_ftrs.drop(columns=['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5'])
str_ftrs = stores.merge(train_ftrs, on=['Store'], how='inner')
train_data = train.merge(str_ftrs,on=['Store','WeekNo','IsHoliday','Year'], how='inner')

Holidays

SuperBowl = np.zeros(len(train_data))
LaborDay = np.zeros(len(train_data))
Thanksgiving = np.zeros(len(train_data))
Christmas = np.zeros(len(train_data))
holidays_df = pd.DataFrame({'SuperBowl':SuperBowl, 'LaborDay':LaborDay, 'Thanksgiving':Thanksgiving,'Christmas':Christmas})
train_data = train_data.join(holidays_df)

train_data.loc[train_data['WeekNo']==6, 'SuperBowl'] = 1
train_data.loc[train_data['WeekNo']==36, 'LaborDay'] = 1
train_data.loc[train_data['WeekNo']==47, 'Thanksgiving'] = 1
train_data.loc[train_data['WeekNo']==52, 'Christmas'] = 1

train_data.Type = [(ord(x)- 64) for x in train_data.Type] # Transformando os valores categóricos (A,B,C) da variável Type em números(1,2,3)

train_data['IsHoliday'] = (train_data['IsHoliday'] == True).astype(int) # Transformando os valores booleanos da variável IsHoliday em números(0,1)

O método usado para selecionar as variaveis mais relevantes foi *Recursive Feature Elimination* utilizando o próprio algoritmo *Random Forest*. A cada iteração o método elimina uma variável, executa o algoritmo e armazena a acurácia encontrada. Ao final das iterações ele determina quais são as variáveis mais relevantes com base na perda de acurácia que cada uma resultou ao ser eliminada.

Vamos saber quais são as variáveis mais relevantes e em seguida fazer um comparativo rodando o *Random Forest* uma vez com todas as variáveis e outra vez somente com as variáveis selecionadas.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE

trees = 5
rfe = RFE(RandomForestRegressor(n_estimators=trees), step=1).fit(train_data.loc[:, train_data.columns != 'Weekly_Sales'], train_data['Weekly_Sales'])
rfe.ranking_
rfe.support_

RF_imp = RandomForestRegressor(n_estimators=trees)
RF_imp.fit(train_data.loc[:, train_data.columns != 'Weekly_Sales'], train_data['Weekly_Sales'])
importance = RF_imp.feature_importances_
xy = pd.DataFrame(
        {'Variables':[(train_data.loc[:, train_data.columns != 'Weekly_Sales'].columns[x]) for x in range(len(importance))],
                      'Importance (%)':importance*100})
xy.insert(2,'RFE',rfe.support_)
feature_importance = xy.sort_values(by=['Importance (%)'], ascending=False)
feature_importance.style.format({'Importance (%)': '{:,.1f}'.format})

Agora vamos ao comparativo:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

def Model(train_data):
    Train, Test = train_test_split(train_data, random_state=1)
    
    X_Train = Train.sort_index().drop(columns=['Weekly_Sales'])
    Y_Train = Train['Weekly_Sales'].sort_index()
    
    X_Test = Test.sort_index().drop(columns=['Weekly_Sales'])
    Y_Test = Test['Weekly_Sales'].sort_index()
    
    # Random Forest Regressor
    print('Calculando')
    trees = 5
    RF = RandomForestRegressor(n_estimators=trees)
    RF.fit(X_Train, Y_Train)
    SalesPrediction = RF.predict(X_Test)
    RF_accuracy = RF.score(X_Test, Y_Test)
    RF_accuracy = metrics.r2_score(Y_Test, SalesPrediction)
    print("Model Accuracy:", round(RF_accuracy*100,1),"%")

    # AVALIAÇÃO WMAE
    soma_SalesPred = 0
    w = np.zeros(len(X_Test.index))
    for l in range(len(X_Test.index)):
        if X_Test['IsHoliday'].iloc[l] == 0:
            w[l] = 1
        else: w[l] = 5
        soma_SalesPred += w[l]*abs(Y_Test.iloc[l] - SalesPrediction[l])
    WMAE = round(soma_SalesPred/np.sum(w),5)
    print('WMAE =',round(WMAE,2))

In [ ]:
print('Com todas as variáveis:')
Model(train_data)

In [ ]:
print('Somente as 8 variáveis mais relevantes:')
train_data_filtered8 = train_data.drop(columns=['Unemployment','Fuel_Price','Christmas','LaborDay','SuperBowl'])
Model(train_data_filtered8)

In [ ]:
print('Somente as 4 variáveis mais relevantes:')
train_data_filtered4 = train_data.loc[:,['Store','Dept','Size','WeekNo','IsHoliday','Weekly_Sales','Year']]
Model(train_data_filtered4)

Houve uma significativa redução do WMAE ao considerar somente as 4 variáveis principais. Ponto positivo para o RFE.

Agora vamos deixar o modelo mais robusto, ajustando os parâmetros do *Random Forest* de maneira a buscar o menor WMAE. Foi utilizada a ferramenta *GridSearchCV* que executa o *Random Forest* para cada combinação dos intervalos predefinidos de parâmetros e, através do método de validação cruzada, encontra os parâmetros que entregam os melhores resultados.

In [ ]:
from sklearn.model_selection import GridSearchCV

# Como comentário pois demora alguns minutos para executar:
#parameters = {'n_estimators':[50,60,70],'min_samples_split':[4,8,16]}
#grid = GridSearchCV(estimator = RandomForestRegressor(), param_grid=parameters, cv=4, n_jobs=-1, verbose=1)                    
#grid.fit(train_data_filtered4.drop(columns=['Weekly_Sales']), train_data_filtered4['Weekly_Sales'])
#grid.best_params_

Obs: como o *dataset* é relativamente grande, a obtenção dos melhores parâmetros é demorada. Então, tentou-se reduzir o número de combinações para agilizar a execução. Com melhor poder computacional e mais tempo, poderíamos encontrar melhores parâmetros.

In [ ]:
def Optmized_Model(train_data):
    Train, Test = train_test_split(train_data, random_state=1)
    
    X_Train = Train.sort_index().drop(columns=['Weekly_Sales'])
    Y_Train = Train['Weekly_Sales'].sort_index()
    
    X_Test = Test.sort_index().drop(columns=['Weekly_Sales'])
    Y_Test = Test['Weekly_Sales'].sort_index()
    
    # Random Forest Regressor
    print('Calculando')
    RF = RandomForestRegressor(n_estimators=70, min_samples_split=4)
    RF.fit(X_Train, Y_Train)
    SalesPrediction = RF.predict(X_Test)
    RF_accuracy = RF.score(X_Test, Y_Test)
    RF_accuracy = metrics.r2_score(Y_Test, SalesPrediction)
    print("Model Accuracy:", round(RF_accuracy*100,1),"%")

    # AVALIAÇÃO WMAE
    soma_SalesPred = 0
    w = np.zeros(len(X_Test.index))
    for l in range(len(X_Test.index)):
        if X_Test['IsHoliday'].iloc[l] == 0:
            w[l] = 1
        else: w[l] = 5
        soma_SalesPred += w[l]*abs(Y_Test.iloc[l] - SalesPrediction[l])
    WMAE = round(soma_SalesPred/np.sum(w),5)
    print('WMAE =',round(WMAE,2))

Rodando com os parâmetros encontrados: 'min_samples_split' = 4 e 'n_estimators' = 70

In [ ]:
Optmized_Model(train_data_filtered4)

Novamente, nota-se que o valor de WMAE diminuiu, indicando bons resultados ao refinar o modelo.

Finalmente, vamos implementá-lo aos dados de teste para submissão do arquivo:

In [ ]:
Train = train.merge(stores, on=['Store']).drop(columns=['Type'])
Train['IsHoliday'] = (Train['IsHoliday'] == True).astype(int)

X_Train = Train.drop(columns=['Weekly_Sales'])
Y_Train = Train['Weekly_Sales']

Test = test.merge(stores, on=['Store']).drop(columns=['Type'])
Test['IsHoliday'] = (Test['IsHoliday'] == True).astype(int)

print('Calculando')
RF_Test = RandomForestRegressor(n_estimators=70, min_samples_split=4)
RF_Test.fit(X_Train, Y_Train)
SalesPrediction_Test = RF_Test.predict(Test)
Weekly_Sales_Test = pd.Series(SalesPrediction_Test, name='Weekly_Sales')
Weekly_Sales_Test.head()

Plotando os resultados no mesmo gráfico de vendas médias por semana, vemos que o modelo adere bem ao comportamento dos dados históricos:

In [ ]:
# Configurando o gráfico:
Weekly_Sales_2010 = train[train.Year==2010]['Weekly_Sales'].groupby(train['WeekNo']).mean()
Weekly_Sales_2011 = train[train.Year==2011]['Weekly_Sales'].groupby(train['WeekNo']).mean()
Weekly_Sales_2012 = train[train.Year==2012]['Weekly_Sales'].groupby(train['WeekNo']).mean()
Sales = Test.join(Weekly_Sales_Test)
Weekly_Sales_Prediction_2012 = Sales[Sales.Year==2012]['Weekly_Sales'].groupby(Test['WeekNo']).mean()
Weekly_Sales_Prediction_2013 = Sales[Sales.Year==2013]['Weekly_Sales'].groupby(Test['WeekNo']).mean()
plt.figure(figsize=(20,7))
sns.lineplot(x=Weekly_Sales_2010.index, y=Weekly_Sales_2010.values)
sns.lineplot(x=Weekly_Sales_2011.index, y=Weekly_Sales_2011.values)
sns.lineplot(x=Weekly_Sales_2012.index, y=Weekly_Sales_2012.values)
sns.lineplot(x=Weekly_Sales_Prediction_2012.index, y=Weekly_Sales_Prediction_2012.values, color='red',linewidth=3.0)
sns.lineplot(x=Weekly_Sales_Prediction_2013.index, y=Weekly_Sales_Prediction_2013.values, color='red',linewidth=3.0)
plt.xticks(range(min(train['WeekNo']), max(train['WeekNo'])+1,1))
plt.legend(['2010', '2011', '2012','Sales Prediction'], loc='best', fontsize=16)
plt.gca().add_artist(plt.legend(['2010', '2011', '2012','Sales Prediction'], loc='upper left', fontsize=16))
plt.title('Sales Prediction\n', fontsize=18)
plt.ylabel('Average Sales', fontsize=16)
plt.xlabel('Week', fontsize=16)
i=0
colors = ['blue', 'lime', 'magenta', 'cyan']
for x in Holidays.iloc[:3,:].drop_duplicates().stack():
    plt.axvline(x, color = colors[i], label = Holidays.columns[i])
    plt.legend(loc='upper center',fontsize=16)
    i = i + 1
plt.show()

Criando o arquivo de submissão:

In [ ]:
Submission_File = pd.read_csv('../input/walmart-recruiting-store-sales-forecasting/sampleSubmission.csv.zip', sep=',')
Submission_File['Weekly_Sales'] = Weekly_Sales_Test
Submission_File.to_csv('Submission_File.csv',index=False)
Submission_File

**Submission score (11Mar2021):** 2802.54183